In [1]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/node-master'       # path to your project on Drive

GIT_USERNAME = "sgsyang" # replace with yours
#GIT_TOKEN = "XXX"           # definitely replace with yours
#GIT_REPOSITORY = "yyy"      # ...nah


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
GIT_PATH = "https://github.com/sgsyang/node.git"

!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/node-master’: File exists
Cloning into 'node'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
README.md
          1,555 100%    0.00k

In [3]:

!pip install category_encoders
!pip install tensorboardX
# !pip install qhoptim

%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import lib
import torch, torch.nn as nn
import torch.nn.functional as F
# from qhoptim.pyt import QHAdam


# DATASETS = {
#     'A9A': fetch_A9A,
#     'EPSILON': fetch_EPSILON,
#     'PROTEIN': fetch_PROTEIN,
#     'YEAR': fetch_YEAR,
#     'HIGGS': fetch_HIGGS,
#     'MICROSOFT': fetch_MICROSOFT,
#     'YAHOO': fetch_YAHOO,
#     'CLICK': fetch_CLICK,
# }



# read the data


device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment_name = 'CLICK_Catoost'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)

  Using cached https://files.pythonhosted.org/packages/35/f1/5843425495765c8c2dd0784a851a93ef204d314fc87bcc2bbb9f662a3ad1/tensorboardX-2.0-py2.py3-none-any.whl
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=0,1
experiment: CLICK_Catoost_2020.05.27_19:10


In [4]:
torch.cuda.get_device_name(device)

'Tesla P100-PCIE-16GB'

##Default Parameters

In [0]:
!pip install catboost
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]

def Test(random_state):
  data = lib.Dataset("CLICK", random_state = random_state, quantile_transform=True, quantile_noise=1e-3)
  num_features = data.X_train.shape[1]
  num_classes = len(set(data.y_train))
  model = CatBoostClassifier(objective = 'CrossEntropy',task_type="GPU",
                           devices='0:1') 
  model.fit(data.X_train, data.y_train)
  pred = model.predict(data.X_test)
  accuracy = accuracy_score(data.y_test, pred)

  error = 1-accuracy
  print('Catboost classification for CLICK')
  print('The Error:',error)
  print(f'Test {random_state} is Over--')
  return error,random_state


score_results = [] 
key_results = []

for i in random_state:
  score, random_state = Test(i)
  score_results.append(score)
  key_results.append(random_state)




     |████████████████████████████████| 64.7MB 43kB/s 


100%|██████████| 115752027/115752027 [00:02<00:00, 56771745.03it/s]


流式输出内容被截断，只能显示最后 5000 行内容。
16:	learn: 0.5541113	total: 183ms	remaining: 10.6s
17:	learn: 0.5497697	total: 193ms	remaining: 10.5s
18:	learn: 0.5457269	total: 203ms	remaining: 10.5s
19:	learn: 0.5395601	total: 230ms	remaining: 11.3s
20:	learn: 0.5362982	total: 240ms	remaining: 11.2s
21:	learn: 0.5330004	total: 250ms	remaining: 11.1s
22:	learn: 0.5302692	total: 261ms	remaining: 11.1s
23:	learn: 0.5255870	total: 271ms	remaining: 11s
24:	learn: 0.5220598	total: 281ms	remaining: 11s
25:	learn: 0.5196027	total: 292ms	remaining: 10.9s
26:	learn: 0.5166036	total: 302ms	remaining: 10.9s
27:	learn: 0.5146971	total: 312ms	remaining: 10.8s
28:	learn: 0.5123660	total: 322ms	remaining: 10.8s
29:	learn: 0.5099804	total: 336ms	remaining: 10.8s
30:	learn: 0.5083892	total: 346ms	remaining: 10.8s
31:	learn: 0.5061309	total: 356ms	remaining: 10.8s
32:	learn: 0.5044183	total: 366ms	remaining: 10.7s
33:	learn: 0.5021229	total: 376ms	remaining: 10.7s
34:	learn: 0.5003744	total: 386ms	remaining: 10.6s
35:	lear

In [0]:
import pprint
from statistics import mean 


Result= dict(zip(key_results,score_results))

pprint.pprint(Result, width=1)


mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

{1: 0.33198000000000005,
 13: 0.33141,
 133: 0.33228,
 295: 0.33233,
 602: 0.33143999999999996,
 970: 0.3307,
 1337: 0.33240000000000003,
 2000: 0.33118000000000003,
 3000: 0.33248,
 10000: 0.3315}
Mean: 0.33177
Max: 0.33248
Min: 0.3307
Distance to max 0.0007099999999999884
Distance to min: -0.0010700000000000154


In [0]:
!pip install catboost

import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


num_features = data.X_train.shape[1]
num_classes = len(set(data.y_train))

model = CatBoostClassifier(objective = 'CrossEntropy',task_type="GPU",
                           devices='0:1') 

model.fit(data.X_train, data.y_train)
pred = model.predict(data.X_test)
accuracy = accuracy_score(data.y_test, pred)
# The classifation Error
# Dataset: CLICK 
# Expected: 0.3461 
print('Catboost classification for CLICK')
print('The Error:',1-accuracy)

0:	learn: 0.6780159	total: 27ms	remaining: 55.3s
1:	learn: 0.6651212	total: 53.9ms	remaining: 55.2s
2:	learn: 0.6539148	total: 80.8ms	remaining: 55.1s
3:	learn: 0.6430368	total: 108ms	remaining: 55.2s
4:	learn: 0.6333910	total: 136ms	remaining: 55.6s
5:	learn: 0.6249514	total: 162ms	remaining: 55.3s
6:	learn: 0.6172037	total: 191ms	remaining: 55.7s
7:	learn: 0.6090739	total: 215ms	remaining: 54.9s
8:	learn: 0.6016605	total: 249ms	remaining: 56.4s
9:	learn: 0.5948414	total: 275ms	remaining: 56s
10:	learn: 0.5881115	total: 300ms	remaining: 55.6s
11:	learn: 0.5821719	total: 325ms	remaining: 55.1s
12:	learn: 0.5768827	total: 352ms	remaining: 55.1s
13:	learn: 0.5705184	total: 375ms	remaining: 54.4s
14:	learn: 0.5647010	total: 398ms	remaining: 54s
15:	learn: 0.5585093	total: 421ms	remaining: 53.5s
16:	learn: 0.5542537	total: 446ms	remaining: 53.2s
17:	learn: 0.5497428	total: 475ms	remaining: 53.5s
18:	learn: 0.5462291	total: 503ms	remaining: 53.7s
19:	learn: 0.5421116	total: 525ms	remaining:

In [0]:
print('Catboost classification for CLICK')
print('The Error:',1-accuracy)

Catboost classification for CLICK
The Error: 0.33214999999999995


##Tuned Parameters



In [5]:
!pip install hyperopt
import xgboost as xgb
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

!pip install catboost
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


# the space 
space={'learning_rate':hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
       'random_strength':hp.quniform("random_strength", 1,20,1),
       'one_hot_max_size':hp.quniform("one_hot_max_size", 0,25,1),
       'l2_leaf_reg': hp.loguniform('l2_leaf_reg',1 ,10),
       'bagging_temperature':hp.uniform('bagging_temperature', 0, 1),
       'leaf_estimation_iterations':hp.quniform("leaf_estimation_iterations", 1,10,1)
    }

     |████████████████████████████████| 64.8MB 44kB/s 


In [0]:
def my_tuning(space):
    reg=CatBoostClassifier( learning_rate=space['learning_rate'],
                          random_strength = int(space['random_strength']),
                          one_hot_max_size = int(space['one_hot_max_size']),
                          l2_leaf_reg = space['l2_leaf_reg'],
                          bagging_temperature = space['bagging_temperature'],
                          leaf_estimation_iterations = int(space['leaf_estimation_iterations']),
                          objective = 'CrossEntropy',task_type="GPU",
                          devices='0:1')
        
    reg.fit(data.X_train, data.y_train,verbose =False)
    pred = reg.predict(data.X_test)

    accuracy = accuracy_score(data.y_test, pred)

    error = 1-accuracy
    # print('Catboost classification for AULDT')
    # print('The Error:',error)
    # print(f'Test {random_state} is Over--')

    #change the metric if you like
    return {'loss':error, 'status': STATUS_OK }


score_results = [] 
key_results = []

random_state = [13, 1, 602, 295, 970, 2000, 3000, 10000]
for i in random_state:
  data = lib.Dataset("CLICK", random_state=i, quantile_transform=True, quantile_noise=1e-3)
  print(f'The dataset with random state {i} is used')

  trials = Trials()
  best = fmin(fn=my_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  # print('Best parameters',best)
  tuned_loss = trials.best_trial['result']['loss']
  print('Best Loss',tuned_loss)
  final = tuned_loss
  print('FINAL Error:',final)
  score_results.append(final)
  key_results.append(i)

100%|██████████| 115752027/115752027 [00:02<00:00, 39455892.17it/s]


The dataset with random state 1337 is used
100%|██████████| 50/50 [08:48<00:00, 10.57s/it, best loss: 0.33115000000000006]
Best Loss 0.33115000000000006
FINAL Error: 0.33115000000000006
The dataset with random state 133 is used
100%|██████████| 50/50 [08:07<00:00,  9.74s/it, best loss: 0.33107]
Best Loss 0.33107
FINAL Error: 0.33107


In [0]:
# Record Experiment
# 1337: 0.33115000000000006
# 133: 0.33107

In [0]:
import pprint
Result= dict(zip(key_results,score_results))

print('----')
pprint.pprint(Result, width=1)

mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

In [0]:
# !pip install hyperopt
# import xgboost as xgb
# import hyperopt
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [0]:
# import math
# print(math.exp(5))

148.4131591025766


In [0]:
# import math

# space={'eta': hp.loguniform('eta',math.exp(-7) ,1),
#        'max_depth': hp.quniform("max_depth", 2,10,2),
#        'subsample': hp.uniform('subsample', 0.5, 1),
#        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
#        'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.5,1),
#        'min_child_weight' : hp.loguniform('min_child_weight', math.exp(-16),math.exp(5)),
#        'alpha' : hp.choice('alpha', [0,hp.loguniform('a',math.exp(-16),math.exp(2))]),
#        'lambda': hp.choice('lambda',[ 0,hp.loguniform('l',math.exp(-16),math.exp(2))]),
#        'gamma': hp.choice ('gamma', [0,hp.loguniform('g',math.exp(-16),math.exp(2))])
        
        
#     }


# # Regression: 
# def hyperparameter_tuning(space):
#     reg=xgb.XGBRegressor( eta = space['eta'],max_depth = int(space['max_depth']), subsample=space['subsample'],
#                          gamma = space['gamma'],reg_alpha = space['alpha'],reg_lambda = space['lambda'],
#                         #  min_child_weight=space['min_child_weight'],
#                          colsample_bytree=space['colsample_bytree'],
#                          colsample_bylevel = space['colsample_bylevel'])
    
#     evaluation = [( data.X_train, data.y_train), ( data.X_valid, data.y_valid)]
    
#     reg.fit(data.X_train, data.y_train,
#             eval_set=evaluation, eval_metric="rmse",
#             early_stopping_rounds=10,verbose=False)

#     pred = reg.predict(data.X_valid)
#     mse= mean_squared_error(data.y_valid, pred)
#     print ("SCORE:", mse)
#     #change the metric if you like
#     return {'loss':mse, 'status': STATUS_OK }

In [0]:
# trials = Trials()
# best = fmin(fn=hyperparameter_tuning,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=50,
#             trials=trials)

# print (best)

[16:15:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6822184
[16:23:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6917804
[16:29:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.69277877
[16:33:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6898674
[16:39:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6913441
[16:46:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.68847024
[16:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:


KeyboardInterrupt: ignored